# Scrapping Website tirto.id 
---
by: __Bakhtiar Amaludin__ | Digital Talent 2018 @ Univ. Gadjah Mada | maziyank@gmail.com | @maziyank
***

Notebook ini adalah hasil dari kegiatan latihan _web scraping_ berita yang ada dalam website tirto.id. _Scraping_ dilakukan melalui halaman indeks berita dengan pembatasan pada jumlah berita terakhit. Berikut adalah beberapa informasi umum terkait _web scraping_ ini:

- Portal Berita (sumber) : tirto.id (URL: https://tirto.id/indeks/)
- Parameter              : Limit Latest News
- Data yang diambil      : Judul, Isi, Tanggal, Author, Gambar
- Format _output_        : .json
- Penyimpanan Image      : ./images/

_libraries_ yang digunakan:
- urllib                 : untuk mengambil data dari internet
- BeautifulSoup          : untuk melakukan parsing dan ekstraksi data halaman web
- datetime               : untuk melakukan rutin-rutin yang berhubungan dengan tanggal
- json                   : untuk menyimpan output ke dalam file .json

## Step 1: Inisiasi libraries

In [1]:
import json
import urllib
from bs4 import BeautifulSoup

hdr = { 'User-Agent': 'My User Agent 2.0',
        'From': 'youremails@domain.com'  }

## Step 2: Membuat Fungsi Untuk Mendapatkan URL Berita dari Indeks
---
Sebelum melakukan ekstraksi data berita, terlebih dahulu dilakukan pengumpulan links dari halaman indeks pada website yang akan discrap. Nah, fungsi ini digunakan untuk mendapatkan semua url dari berita yang ada dalam halaman indeks tersebut. Kebetulan website tirto ini sudah memiliki api untuk menampilkan list seluruh berita terakhir.

In [2]:
def getLinks(limit):    
    url = 'https://tirto.id/apis/articlef/latest/all?page=1&limit=' + str(limit)    

    request = urllib.request.Request(url, headers=hdr)
    response = urllib.request.urlopen(request)
    
    data = json.loads(response.read())    
    with open('tirto_links.json', 'w') as outfile:  
        json.dump(data, outfile)        
    
    return data

## Step 3: Membuat Fungsi Untuk Membaca Berita

Dari _link single page_ berita yang telah diperoleh, langkah berikutnya adalah mengekstrasi data dari halaman yang diperoleh dari _link_ tersebut. Fungsi ini dibuat untuk melakukan tugas tersebut

In [3]:
def getBerita(sublink):
    url = 'https://tirto.id' + str(sublink)    
    
    request = urllib.request.Request(url, headers=hdr)
    response = urllib.request.urlopen(request)    
    
    soup = BeautifulSoup(response.read(),"html.parser")      

    content = json.loads(soup.find_all('script')[6].text.replace('window.__NUXT__=','')[:-1])['data'][0]['article']
    
     # mengambil judul dari artikel     
    judul = content['judul']

    # mengambil tanggal dari artikel     
    tanggal = content['date_news']
    
    # membersikan isi berita
    isi = content['isi']

    # mengambil url gambar dari artikel     
    gambar = 'https://mmc.tirto.id/image/otf/700x0/' + content['image'][0]['url']

    # mengambil nama redaksi dari artikel     
    author = content['author'][0]['nama_author']
    
    # simpan image berita ke folder images
    opener = urllib.request.build_opener()
    opener.addheaders = [('User-agent', 'Mozilla/5.0')]
    urllib.request.install_opener(opener)
    urllib.request.urlretrieve(gambar, "images/"+content['image'][0]['nama_file'])    
            
    return {"judul": judul, "isi": isi, "tanggal":tanggal, "author":author, "gambar": gambar }    


## Step 4: Mulai Scraping 

Proses _web scraping_ dilakukan pada tahap ini menggunakan dua fungsi yang telah dibuat sebelumnya.

In [4]:
berita = []
limit = 1

# mendaptkan seluruh links sesuai parameter seberlumnya
links = getLinks(5)

# ekstraksi data dari setiap link yang telah didapat dan menyimpanya ke variabel berita[]
for link in links['data']:        
    getBerita(link['articleUrl'])
    berita.append(getBerita(link['articleUrl']))
    print('scraping :', link['articleUrl'])
      
# menyimpan hasil scraping ke dalam file
with open('tirto_contents.json', 'w') as outfile:  
    json.dump(berita, outfile)
    print('Scraping Done.. ')



scraping : /winamp-yang-mati-kini-terlahir-kembali-c8nZ
scraping : /karnaval-mandiri-apresiasi-kinerja-karyawan-c8H6
scraping : /hasil-french-open-2018-greysia-apriyani-disingkirkan-ganda-jepang-c8H1
scraping : /jelang-timnas-u-19-indonesia-vs-jepang-egy-todd-siap-tampil-c8H5
scraping : /rembuk-dan-hubungannya-dengan-revolusi-mental-c8H4
Scraping Done.. 
